Download the MNIST dataset for multiclass classification of handwritten digits. Implement the kernel Perceptron from scratch and run it to train 10 binary classifiers, one for each of the 10 digits (one-vs-all encoding), using the polynomial kernel. In order to extract a binary classifier, do the following: run the algorithm for a given number of epochs (e.g., cycles over the entire training data) on a random permutation of the training set and collect the ensemble of predictorsused by the Perceptron when predicting each training datapoint . Then use:

- the average of the predictors in the ensemble;
- the predictor achieving the smallest training error among those in the ensemble.

Based on the given split of the data in training and test set, evaluate the multiclass classification performance (zero-one loss) for different values of the number of epochs (go up to at least 10 epochs) and the degree of the polynomial (go up to at least degree 6). In order to use the 10 binary classifiers to make multiclass predictions, use the fact that binary classifiers trained by the Perceptron have the form _argmax gixi_ and predict using  wherecorresponds to the binary classifier for class.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
#dataset MNIST
dataset = pd.read_csv("mnist_test.csv")
dataset.head(10)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#divide the data domain from the label set 
X = dataset.drop(["label"], axis =1).values

In [4]:
#look at the data domain
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [5]:
#extracting the label set 
Y = dataset[["label"]].values

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [7]:
#train and test split 
np.random.seed(67)
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.4, stratify = Y)

In [8]:
y_train

array([[5],
       [9],
       [6],
       ...,
       [6],
       [0],
       [2]])

In [9]:
x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
#Apply OneHotEncoder
encoder = OneHotEncoder(sparse=False)

In [11]:
#apply the one-hot encoder at the train 
y_train = encoder.fit_transform(y_train)

/Users/margheritamaroni/opt/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Since we have a binary label set {-1, 1}, we want the 0 to become -1.


In [12]:
y_train[y_train == 0] = -1

In [13]:
y_train

array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1.,  1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ..., -1., -1., -1.],
       [ 1., -1., -1., ..., -1., -1., -1.],
       [-1., -1.,  1., ..., -1., -1., -1.]])

In [14]:
y_train[100,:]

array([-1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.])

In [15]:
y_train[100,:].argmax() 
#come farò la predizione in multiclass classification

5

In [16]:
y_train.shape
#6000 observations for 10 columns

(6000, 10)


# Polynomial kernel

In [17]:
(1+np.dot(x_train[0], x_train[0]))**5

4865624920.249177

In [18]:
(1+(x_train[0].T@ x_train[0]))**5

4865624920.249177

In [19]:
#Create the matrix for degree = 5

In [20]:
#inizializza la matrice vuota
Kernel = np.zeros((6000, 6000))
for column in range(6000):
    for row in range(6000):
        Kernel[row,column] = (1+(x_train[row].T@ x_train[column]))**5

In [21]:
Kernel.shape

(6000, 6000)

In [22]:
np.savetxt("Fourth_Matrix.txt", Kernel, delimiter = ",")

In [23]:
matrix_d5= pd.read_csv("Fourth_Matrix.txt", header = None)

In [24]:
matrix_d5 = matrix_d5.to_numpy()

In [25]:
matrix_d5[:,1]

array([1.36125597e+07, 5.67297363e+09, 5.47130035e+07, ...,
       9.95379052e+07, 1.06525568e+09, 1.60956270e+08])

In [26]:
matrix_d5[1,:]

array([1.36125597e+07, 5.67297363e+09, 5.47130035e+07, ...,
       9.95379052e+07, 1.06525568e+09, 1.60956270e+08])

In [27]:
matrix_d5[:,1]

array([1.36125597e+07, 5.67297363e+09, 5.47130035e+07, ...,
       9.95379052e+07, 1.06525568e+09, 1.60956270e+08])

In [28]:
y_train[:,0],len(y_train[:,0])

(array([-1., -1., -1., ..., -1.,  1., -1.]), 6000)

In [29]:
matrix_d5[:1] * y_train[:,0] 

array([[-4.86562492e+09, -1.36125597e+07, -2.28542421e+08, ...,
        -2.44662374e+08,  5.90595583e+07, -7.07033704e+07]])

In [30]:
np.sum(matrix_d5[:,1] * y_train[:,0])

-686574925580.8176

In [31]:
np.sign(np.sum(matrix_d5[:,1] * y_train[:,0]))

-1.0

In [32]:
#Five epochs
S5 =np.zeros((6000,10))
for digit in range(10):
    for epoch in range(5):
        y_pred = np.zeros(6000)
        for point in range(6000):
            y_pred[point]= np.sign(np.sum(matrix_d5[:,point] * y_train[:,digit] * S5[:,digit]))

        for point in range(6000):
            if y_pred[point] != y_train[point,digit]:
                S5[point,digit]+=1

In [33]:
np.savetxt("S5_d5_e5.txt", S5,  delimiter = ",")

In [34]:
S5_d5_e5 = pd.read_csv("S5_d5_e5.txt", header = None)
S5_d5_e5

,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...
5995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0
5996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
5997,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
5998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [35]:
np.unique(S5[:,0],return_counts = True)

(array([1., 2., 3., 4., 5.]), array([5747,  135,   40,   23,   55]))

In [36]:
#Prediction for the training set
pred = np.zeros((6000,10))
for digit in range(10):
    for point in range(6000):
        pred[point,digit] = np.sign(np.sum(matrix_d5[:,point]*y_train[:,digit]*S5[:,digit]))


In [37]:
pred[:,0]

array([-1., -1., -1., ..., -1.,  1., -1.])

In [38]:
y_train[:,0]

array([-1., -1., -1., ..., -1.,  1., -1.])

we need to look if these two columns correspond


In [39]:
matrix_training_error = np.zeros((6000,10))

In [40]:
for point in range(6000):
    for digit in range(10):
        if pred[point,digit] != y_train[point,digit]:
            matrix_training_error[point, digit] = 1

In [41]:
matrix_training_error

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [42]:
matrix_training_error[:,0]

array([0., 0., 0., ..., 0., 0., 0.])

I'm going to compute the training error for each digit of my matrix of degree = 2 for 10 epochs

In [43]:
a = np.mean(matrix_training_error[:,0])
a

0.0125

In [44]:
b = np.mean(matrix_training_error[:,1])
b

0.014333333333333333

In [45]:
c = np.mean(matrix_training_error[:,2])
c

0.0265

In [46]:
d = np.mean(matrix_training_error[:,3])
d

0.028166666666666666

In [47]:
e = np.mean(matrix_training_error[:,4])
e

0.03766666666666667

In [48]:
f = np.mean(matrix_training_error[:,5])
f

0.04816666666666667

In [49]:
g = np.mean(matrix_training_error[:,6])
g

0.016833333333333332

In [50]:
h = np.mean(matrix_training_error[:,7])
h

0.030833333333333334

In [51]:
i = np.mean(matrix_training_error[:,8])
i 

0.039

In [52]:
j = np.mean(matrix_training_error[:,9])
j

0.06116666666666667

In [53]:
#Training error for 5 epochs for 10 digits
five_epochs = np.array([a,b,c,d,e,f,g,h,i,j])
five_epochs

array([0.0125    , 0.01433333, 0.0265    , 0.02816667, 0.03766667,
       0.04816667, 0.01683333, 0.03083333, 0.039     , 0.06116667])

In [54]:
#Ten epochs
S10 =np.zeros((6000,10))
for epoch in range(10):
    for digit in range(10):
        y_pred = np.zeros(6000)
        for point in range(6000):
            y_pred[point]= np.sign(np.sum(matrix_d5[:,point] * y_train[:,digit] * S10[:,digit]))

        for point in range(6000):
            if y_pred[point] != y_train[point,digit]:
                S10[point,digit]+=1

In [55]:
np.savetxt("S10_d5_e10.txt", S10, delimiter = ',')

In [56]:
S10_d5_e10 = pd.read_csv("S10_d5_e10.txt", header = None)
S10_d5_e10

,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,1.0,5.0,1.0,6.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,7.0,1.0,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...
5995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0
5996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
5997,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
5998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [57]:
pred = np.zeros((6000,10))
for digit in range(10):
    for point in range(6000):
        pred[point,digit] = np.sign(np.sum(matrix_d5[:,point]*y_train[:,digit]*S10[:,digit]))

In [58]:
matrix_training_error = np.zeros((6000,10))

In [59]:
for point in range(6000):
    for digit in range(10):
        if pred[point,digit] != y_train[point,digit]:
            matrix_training_error[point,digit] = 1

In [60]:
ten_epochs = []
for numero in range(10):
    ten_epochs.append(np.mean(matrix_training_error[:,numero]))

In [61]:
ten_epochs

[0.007833333333333333,
 0.0075,
 0.016666666666666666,
 0.020166666666666666,
 0.023333333333333334,
 0.021333333333333333,
 0.012166666666666666,
 0.019833333333333335,
 0.019333333333333334,
 0.035666666666666666]

In [62]:
ten_epochs = np.array(ten_epochs)

In [63]:
#Training error for 10 epochs and 10 digits
ten_epochs

array([0.00783333, 0.0075    , 0.01666667, 0.02016667, 0.02333333,
       0.02133333, 0.01216667, 0.01983333, 0.01933333, 0.03566667])

In [64]:
#Fifteen epochs
S15 =np.zeros((6000,10))
for epoch in range(15):
    for digit in range(10):
        y_pred = np.zeros(6000)
        for point in range(6000):
            y_pred[point]= np.sign(np.sum(matrix_d5[:,point] * y_train[:,digit] * S15[:,digit]))

        for point in range(6000):
            if y_pred[point] != y_train[point,digit]:
                S15[point,digit]+=1

In [65]:
np.savetxt("S15_d5_e15.txt", S15, delimiter = ',')

In [66]:
S15_d5_e15 = pd.read_csv("S15_d5_e15.txt", header = None)
S15_d5_e15

,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,1.0,5.0,1.0,6.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,12.0,1.0,15.0,1.0
...,...,...,...,...,...,...,...,...,...,...
5995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0
5996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
5997,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
5998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [67]:
pred = np.zeros((6000,10))
for digit in range(10):
    for point in range(6000):
        pred[point,digit] = np.sign(np.sum(matrix_d5[:,point]*y_train[:,digit]*S15[:,digit]))

In [68]:
matrix_training_error = np.zeros((6000,10))

In [69]:
for point in range(6000):
    for digit in range(10):
        if pred[point,digit] != y_train[point,digit]:
            matrix_training_error[point,digit] = 1

In [70]:
fifteen_epochs = []
for numero in range(10):
    fifteen_epochs.append(np.mean(matrix_training_error[:,numero]))

In [71]:
fifteen_epochs = np.array(fifteen_epochs)

In [72]:
#Training error for 15 epochs and 10 digits
fifteen_epochs

array([0.00616667, 0.00483333, 0.012     , 0.0155    , 0.01783333,
       0.0145    , 0.00866667, 0.0155    , 0.0145    , 0.02833333])

In [73]:
#Twenty epochs
S20 =np.zeros((6000,10))
for epoch in range(20):
    for digit in range(10):
        y_pred = np.zeros(6000)
        for point in range(6000):
            y_pred[point]= np.sign(np.sum(matrix_d5[:,point] * y_train[:,digit] * S20[:,digit]))

        for point in range(6000):
            if y_pred[point] != y_train[point,digit]:
                S20[point,digit]+=1

In [74]:
np.savetxt("S20_d5_e20.txt",S20, delimiter = ',')

In [75]:
S20_d5_e20 = pd.read_csv("S20_d5_e20.txt", header = None)
S20_d5_e20

,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,1.0,5.0,1.0,6.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,17.0,1.0,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...
5995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0
5996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
5997,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
5998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [76]:
pred = np.zeros((6000,10))
for digit in range(10):
    for point in range(6000):
        pred[point,digit] = np.sign(np.sum(matrix_d5[:,point]*y_train[:,digit]*S20[:,digit]))

In [77]:
matrix_training_error = np.zeros((6000,10))

In [78]:
for point in range(6000):
    for digit in range(10):
        if pred[point,digit] != y_train[point,digit]:
            matrix_training_error[point,digit] = 1

In [79]:
twenty_epochs = []
for numero in range(10):
    twenty_epochs.append(np.mean(matrix_training_error[:,numero]))

In [80]:
twenty_epochs = np.array(twenty_epochs)

In [81]:
#Training error for 20 epochs and 10 digits
twenty_epochs

array([0.00416667, 0.0045    , 0.0105    , 0.01      , 0.01283333,
       0.00983333, 0.00633333, 0.0125    , 0.007     , 0.01933333])

In [82]:
#Twenty_five epochs
S25 =np.zeros((6000,10))
for epoch in range(25):
    for digit in range(10):
        y_pred = np.zeros(6000)
        for point in range(6000):
            y_pred[point]= np.sign(np.sum(matrix_d5[:,point] * y_train[:,digit] * S25[:,digit]))

        for point in range(6000):
            if y_pred[point] != y_train[point,digit]:
                S25[point,digit]+=1

In [83]:
np.savetxt("S25_d5_e25.txt", S25, delimiter = ',')

In [84]:
S25_d5_e25 = pd.read_csv("S25_d5_e25.txt", header = None)
S25_d5_e25

,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,1.0,5.0,1.0,6.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,22.0,1.0,25.0,1.0
...,...,...,...,...,...,...,...,...,...,...
5995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0
5996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
5997,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
5998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [85]:
pred = np.zeros((6000,10))
for digit in range(10):
    for point in range(6000):
        pred[point,digit] = np.sign(np.sum(matrix_d5[:,point]*y_train[:,digit]*S25[:,digit]))

In [86]:
matrix_training_error = np.zeros((6000,10))

In [87]:
for point in range(6000):
    for digit in range(10):
        if pred[point,digit] != y_train[point,digit]:
            matrix_training_error[point,digit] = 1

In [88]:
twenty_five_epochs = []
for numero in range(10):
    twenty_five_epochs.append(np.mean(matrix_training_error[:,numero]))

In [89]:
twenty_five_epochs = np.array(twenty_five_epochs)

In [90]:
#training error for 25 epochs and 10 digits

In [91]:
twenty_five_epochs

array([0.004     , 0.00416667, 0.00766667, 0.00766667, 0.00866667,
       0.0065    , 0.00516667, 0.01      , 0.00466667, 0.01566667])

In [92]:
rows= np.array([five_epochs, ten_epochs, fifteen_epochs, twenty_epochs, twenty_five_epochs])

In [93]:
index = ["five_epochs", "ten_epochs", "fifteen_epochs", "twenty_epochs", "twenty_five_epochs"] 

In [94]:
df = pd.DataFrame(rows, index = index)

In [95]:
df

,0,1,2,3,4,5,6,7,8,9
five_epochs,0.012500,0.014333,0.026500,0.028167,0.037667,0.048167,0.016833,0.030833,0.039000,0.061167
ten_epochs,0.007833,0.007500,0.016667,0.020167,0.023333,0.021333,0.012167,0.019833,0.019333,0.035667
fifteen_epochs,0.006167,0.004833,0.012000,0.015500,0.017833,0.014500,0.008667,0.015500,0.014500,0.028333
twenty_epochs,0.004167,0.004500,0.010500,0.010000,0.012833,0.009833,0.006333,0.012500,0.007000,0.019333
twenty_five_epochs,0.004000,0.004167,0.007667,0.007667,0.008667,0.006500,0.005167,0.010000,0.004667,0.015667


**Salvo la matrice con tutti i training error per le 25 epoche, degree = 2**

In [96]:
np.savetxt("training_error_degree_5.csv", df, delimiter = ",")

In [97]:
degree_5 = pd.read_csv("training_error_degree_5.csv", header = None)

In [98]:
degree_5

,0,1,2,3,4,5,6,7,8,9
0,0.012500,0.014333,0.026500,0.028167,0.037667,0.048167,0.016833,0.030833,0.039000,0.061167
1,0.007833,0.007500,0.016667,0.020167,0.023333,0.021333,0.012167,0.019833,0.019333,0.035667
2,0.006167,0.004833,0.012000,0.015500,0.017833,0.014500,0.008667,0.015500,0.014500,0.028333
3,0.004167,0.004500,0.010500,0.010000,0.012833,0.009833,0.006333,0.012500,0.007000,0.019333
4,0.004000,0.004167,0.007667,0.007667,0.008667,0.006500,0.005167,0.010000,0.004667,0.015667


In [99]:
import matplotlib.pyplot as plt
plt.plot(degree_5)
plt.grid()
plt.show()

<Figure size 640x480 with 1 Axes>

In [100]:
degree_5.iloc[:,0].values.argmin()

4

In [101]:
degree_5.iloc[:,1].values.argmin()

4

In [102]:
degree_5.iloc[:,2].values.argmin()

4

In [103]:
degree_5.iloc[:,3].values.argmin()

4

In [104]:
degree_5.iloc[:,4].values.argmin()

4

In [105]:
degree_5.iloc[:,5].values.argmin()

4

In [106]:
degree_5.iloc[:,6].values.argmin()

4

In [107]:
degree_5.iloc[:,7].values.argmin()

4

In [108]:
degree_5.iloc[:,8].values.argmin()

4

In [109]:
degree_5.iloc[:,9].values.argmin()

4

In [110]:
#25 epoch give the smallest training error for each digit
#S25 is the best predictor for each column (=digit)

# TEST 

In [111]:
test_error = np.zeros(4000)

In [112]:
for point in range(4000):
    y_pred = np.zeros(10)
    for digit in range(10):
        y_pred[digit] = np.sign(np.sum((1+np.dot(x_train[:,:], x_test[point,:]))**5
                *y_train[:,digit]*S25[:,digit]))
    if y_pred.argmax() != y_test[point]:
        test_error[point] = 1 #zero-one loss

In [113]:
np.mean(test_error)
#9.75% test error 

0.0975

In [114]:
S5[:]

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 5.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 2., ..., 1., 1., 1.]])

In [115]:
#Create a new predictor that is the average of the five previous predictors
average = np.zeros((6000,10))
average = average + np.array(S5[:]+S10[:]+S15[:]+S20[:]+S25[:]) 
average /= 5

In [116]:
average

array([[1. , 1. , 1. , ..., 1. , 1. , 1. ],
       [1. , 1. , 1. , ..., 1. , 1. , 6.6],
       [1. , 1. , 1. , ..., 1. , 1. , 1. ],
       ...,
       [1. , 1. , 1. , ..., 1. , 1. , 1. ],
       [1. , 1. , 1. , ..., 1. , 1. , 1. ],
       [1. , 1. , 2. , ..., 1. , 1. , 1. ]])

In [117]:
np.savetxt("average_d5.txt", average, delimiter = ',')
pd.read_csv("average_d5.txt", header = None)

,0,1,2,3,4,5,6,7,8,9
0,1.0,1.0,1.0,4.2,1.0,5.8,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,6.6
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
3,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,12.0,1.0,15.0,1.0
...,...,...,...,...,...,...,...,...,...,...
5995,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0
5996,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0
5997,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
5998,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [118]:
test_error = np.zeros(4000)
for point in range(4000):
    y_pred = np.zeros(10)
    for digit in range(10):
        y_pred[digit] = np.sign(np.sum((1+np.dot(x_train[:,:], x_test[point,:]))**5
                *y_train[:,digit]*average[:,digit]))
    if y_pred.argmax() != y_test[point]:
        test_error[point] = 1 #zero-one loss

In [119]:
np.mean(test_error)
#13.4% test error

0.134